In [12]:
import multiprocessing
import os
import numpy as np
import json
import math
import threading

In [13]:
movies_path = os.path.join(os.path.join(os.getcwd(),'Datasets'),'movies_dataset')

user_based_dict_path = os.path.join(movies_path,'utility_matrix_user_based.txt')
item_based_dict_path = os.path.join(movies_path,'utility_matrix_item_based.txt')


CORES = multiprocessing.cpu_count()
cliques = {}

In [14]:
with open(user_based_dict_path) as fr:
    user_based_utility = json.load(fr)
    
'''
with open(user_based_dict_path) as fr:
    item_based_utility = json.load(fr)
'''    

'\nwith open(user_based_dict_path) as fr:\n    item_based_utility = json.load(fr)\n'

In [15]:
user_based_correct = {}

for key1 in user_based_utility.keys():
    for key2 in user_based_utility[key1].keys():

        key11 = float(key1)
        key22 = float(key2)

        if(user_based_correct.get(key11) == None):
            user_based_correct[key11] = {}
        
        user_based_correct[key11][key22] = float(user_based_utility[key1][key2])

user_based_utility = user_based_correct

In [6]:
# items are casted to float

items = np.array(list(item_based_utility.keys())).astype(np.int)

items

NameError: name 'item_based_utility' is not defined

In [16]:
# divide the dataset in 4 folds to perform 4-folds cross validation

num_folds = 4
folds = [{}]*num_folds

for key1 in user_based_utility.keys():
    for key2 in user_based_utility[key1].keys():
        rand_index = np.random.randint(num_folds)
        
        tmp_dict = folds[rand_index]

        if(tmp_dict.get(key1) == None):
            tmp_dict[key1] = {}
        
        tmp_dict[key1][key2] = user_based_utility[key1][key2]
    

In [17]:
def merge_dicts(dicts):
    ret = {}

    for i in range(len(dicts)):
        tmp_dict = dicts[i]
        for key1 in tmp_dict.keys():
            for key2 in tmp_dict[key1].keys():

                if(ret.get(key1)==None):
                    ret[key1] = {}
                ret[key1][key2] = tmp_dict[key1][key2]

    return ret

In [18]:
train_dict = merge_dicts([fold for fold in folds[:3]])
test_dict = folds[3] 

In [19]:
"""
    Calculates the average value of the 
    valeues in a

    a -> dictionary of non zero values 
"""
def avg(a):
    i=0
    tot = 0
    for k in a.keys():
        i +=1
        tot += a.get(k)
    
    return tot/i

"""
    Calculates a new dictionary that for each
    non null value of a has that values - const

    returns a - const

    a -> dictioary of non zero values
"""
def scale(a,const):
    ret = {}
    
    for k in a.keys():
        ret[k] = a.get(k) - const
    
    return ret

"""
    calculates the norm of a dictionary

    a -> dictioary of non zero values
"""
def norm (a):
    tot = 0
    for k in a.keys():
        tot += pow(a.get(k),2) 

    return math.sqrt(tot)

"""
    computes the inner product of two dictionaries

    a -> dictioary of non zero values
    b -> dictioary of non zero values

"""
def inner_product(a,b):
    tot = 0
    for k in a.keys():
        b_tmp = b.get(k)
        if(b_tmp != None):
            tot += b_tmp * a.get(k) 

    return tot

In [20]:
def compute_correlation_coefficent(a,b, is_explicit):

    a_scaled = a
    b_scaled = b

    if(is_explicit):
        avg_a = avg(a)
        avg_b = avg(b)

        a_scaled = scale(a, avg_a)
        b_scaled = scale(b, avg_b)

    a_scaled_norm = norm(a_scaled)
    b_scaled_norm = norm(b_scaled)

    if(a_scaled_norm == 0 or b_scaled_norm == 0):
        print('One of the two vectors has 0 norm, returning 0.')
        return 0 

    sim = inner_product(a_scaled,b_scaled)/(a_scaled_norm*b_scaled_norm)

    return sim

In [39]:
class ComputeCorrelationCoefficentThread (threading.Thread):
   def __init__(self, user_dict, user_ids, utility_matrix, is_explicit, name = None):
      threading.Thread.__init__(self)
      self.name = name
      self.user_dict = user_dict
      self.user_ids = user_ids
      self.is_explicit = is_explicit
      self.utility_matrix = utility_matrix
      self.result = None
   
   def run(self):
      if self.name == None:
         raise Exception('Something bad happened, thread has no name')
      print('Thread {} started'.format(self.name))
      self.compute_similarities()

   
   def compute_similarities(self):
      similarities = []
      for user in self.user_ids:
         tmp_user_dict = self.utility_matrix[user]
         similarity = compute_correlation_coefficent(self.user_dict, tmp_user_dict, self.is_explicit)
         similarities.append([user,similarity])
      
      self.result = similarities

   # vector of similarities --> [userID, sim_score]
   def join(self):
        threading.Thread.join(self)
        if self.result is not None:
            return self.result
        else:
            print('Error using threads, result is None')

In [ ]:
def make_partitions(num_folds, data):
    data_size = len(data)
    fold_size = data_size // num_folds
    partitions = []
    for idx in range(num_folds-1):
        partitions.append(data[idx*fold_size:(idx+1)*fold_size])
    partitions.append(data[idx*fold_size:])
    return partitions

In [41]:
"""
    Returns the prediction of the rating
    that the user x cuold give to item i

    x -> userID
    explicit -> explicit dictionary of all the users
        explicit[user][item] = rating from that user to that item
        if present
    i -> itemID
    k -> number of users to return
"""
def compute_clique_without_implicit(user, utility_matrix, clique_size):

    unique_users = list(set(utility_matrix.keys()))
    user_dict = utility_matrix[user]
    partitions = make_partitions(CORES,unique_users)    
        

    threads = []
    for idx in range(CORES):
        name = "Thread-{}".format(idx)
        thread = ComputeCorrelationCoefficentThread(name = name, 
                                                user_dict = user_dict, 
                                                user_ids = partitions[idx], 
                                                utility_matrix = utility_matrix, 
                                                is_explicit = True)
        threads.append(thread)

    
    [thread.start() for thread in threads]    
    print('======= all threads started =======')

    results = [thread.join() for thread in threads]
    print('======= all threads joined =======')
    
    similarities = []
    for elem in results:
        for similarity in elem:
            similarities.append(similarity)
    
    similarities = np.array(similarities)
    clique = similarities[np.argsort(similarities[:,1])[::-1]][1:clique_size+1]

    return clique

In [60]:
def predict(user, item, clique, utility_matrix):
    numerator = 0
    denominator = 0
    for elem in clique:
        neighbor = elem[0]
        similarity = elem[1]

        neigh_dict = utility_matrix[neighbor]
        rating = neigh_dict.get(item)
        
        if  rating == None:
            continue

        numerator += rating*similarity
        denominator += similarity
    
    if denominator == 0:
        print('denominator is 0, no user in the clique rated this item')
        return -1
    
    return numerator/denominator
        

In [42]:
user = list(test_dict.keys())[0]    
print('user:{}'.format(user))

clique = compute_clique_without_implicit(user, train_dict, 100)

 of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returning 0.One of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returning 0.

One of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returning 0.One of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returning 0.

One of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returning 0.
One of the two vectors has 0 norm, returnin

array([[2.29107000e+05, 5.80643959e-01],
       [2.29107000e+05, 5.80643959e-01],
       [7.93300000e+03, 5.35223358e-01],
       [2.18291000e+05, 5.19934964e-01],
       [4.23260000e+04, 4.39534830e-01],
       [1.20245000e+05, 4.38171981e-01],
       [6.89000000e+02, 4.31570006e-01],
       [8.41100000e+04, 4.15996443e-01],
       [7.36170000e+04, 4.15614400e-01],
       [5.15120000e+04, 4.02929549e-01],
       [9.59930000e+04, 4.00771864e-01],
       [1.66010000e+04, 4.00687978e-01],
       [8.88570000e+04, 3.99241729e-01],
       [2.17654000e+05, 3.92728827e-01],
       [1.48878000e+05, 3.92728827e-01],
       [1.71786000e+05, 3.91918863e-01],
       [1.86607000e+05, 3.91801163e-01],
       [3.11140000e+04, 3.90889984e-01],
       [1.29480000e+04, 3.86653315e-01],
       [3.63400000e+04, 3.85222346e-01],
       [2.23054000e+05, 3.84993651e-01],
       [1.06155000e+05, 3.82460369e-01],
       [1.88889000e+05, 3.80178631e-01],
       [1.91691000e+05, 3.74059304e-01],
       [3.910800

In [66]:
test_items = list(test_dict[user].keys())

predictions = np.array([ [item, predict(user, item, clique, train_dict), test_dict[user][item]] for item in test_items])
print(predictions)

denominator is 0, no user in the clique rated this item
denominator is 0, no user in the clique rated this item
denominator is 0, no user in the clique rated this item
denominator is 0, no user in the clique rated this item
denominator is 0, no user in the clique rated this item
denominator is 0, no user in the clique rated this item
[[ 3.37940000e+04  4.25298928e+00  4.00000000e+00]
 [ 9.88090000e+04  1.97262313e+00  5.00000000e-01]
 [ 1.22100000e+03  4.66434470e+00  5.00000000e+00]
 [ 6.83580000e+04  5.00000000e+00  5.00000000e+00]
 [ 5.57700000e+03 -1.00000000e+00  5.00000000e+00]
 [ 2.76200000e+03  4.44735375e+00  4.50000000e+00]
 [ 4.22600000e+03  4.51616960e+00  4.00000000e+00]
 [ 4.87800000e+03  5.00000000e+00  5.00000000e+00]
 [ 2.95900000e+03  4.63784546e+00  4.00000000e+00]
 [ 1.47000000e+02 -1.00000000e+00  4.50000000e+00]
 [ 6.98440000e+04  5.00000000e+00  5.00000000e+00]
 [ 9.15420000e+04 -1.00000000e+00  5.00000000e+00]
 [ 9.24390000e+04 -1.00000000e+00  5.00000000e+00]
 